In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import plotly.express as px

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, 
    QuadraticDiscriminantAnalysis
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import (
    load_iris,
    load_wine,
    load_breast_cancer
)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    StandardScaler,
    LabelEncoder, 
    OneHotEncoder,
)
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score,
    classification_report,
    confusion_matrix
)
from sklearn.compose import ColumnTransformer, make_column_transformer

In [ ]:
from scipy import stats
from scipy.interpolate import interp1d

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
images_path = PATH / 'images'
MODELS = PATH / 'models'
images_path.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'
ad_click_path = PATH / 'advertising.csv'

In [ ]:
model_path = MODELS / 'simple_nn_1_hidden_128_sigmoid.ckpt'

## Initialize the model

In [ ]:
class SimpleNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, dropout_prob=0.5):
        super().__init__()
        # Define layers
        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=dropout_prob)
        self.output = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        # Flatten the image tensors
        x = x.view(x.size(0), -1)
        # Hidden layer with ReLU activation
        h = self.hidden(x)
        z = self.relu(h)

        return z

    def run_logits(self, x):
        # Flatten the image tensors
        h = self.encode(x)

        # Apply dropout
        # h = self.dropout(h)

        # Output layer with Sigmoid activation
        r = self.output(h)

        return r

    def regress_vec(self, z):
        l = self.output(z)
        p = self.sigmoid(l)

        return p

    def run_inference(self, x):
        r = self.run_logits(x)

        y_hat = self.sigmoid(r)
        
        return y_hat
      
    def forward(self, x):
        r = self.run_inference(x)
        
        return r

In [ ]:
class InferenceModel(object):

    def __init__(self, model):
        self.model = model.eval()

    @torch.inference_mode()
    def run_encode(self, x):
        return self.model.encode(x)

    @torch.inference_mode()
    def inference_np(self, x):
        y_hat = self.model.run_inference(x)
        y_np = y_hat.cpu().detach().numpy()

        return y_np

    @torch.inference_mode()
    def encode_np(self, x):
        z = self.run_encode(x)
        z_np = z.cpu().detach().numpy()

        return z_np

    @torch.inference_mode()
    def regress_vec(self, z):
        return self.model.regress_vec(z)

    @torch.inference_mode()
    def regress_np(self, z):
        p = self.regress_vec(z)
        p_np = p.cpu().detach().numpy()

        return p_np

    def __call__(*args, **kwargs):
        return self.run(*args, **kwargs)

In [ ]:
def inference_ds(ds):
    with tqdm(ds) as prds:
        zs = np.array(
            [model.encode_np(x) for x, _ in prds]
        )

    return zs

In [ ]:
def find_zs(ds):
    z_kl = dict()
    x_kl = dict()
    z_ks = dict()
    x_ks = dict()
    with tqdm(ds) as prds:
        for x, y in prds:
            z_k = model.encode_np(x)
            z_kl.setdefault(y, list())
            x_kl.setdefault(y, list())
            z_kl[y].append(z_k)
            x_kl[y].append(x)
    for k, v in z_kl.items():
        z_ks[k] = np.array(v)
        x_ks[k] = x_kl[k]

    return x_ks, z_ks

In [ ]:
def gr_idx(z, zs):
    with tqdm(zs) as przs:
        gr = np.array(
            [i for i, z_s in enumerate(przs) if (z <= z_s).all()]
        )

    return gr

## Load the model

In [ ]:
pl_state_dict = torch.load(model_path, map_location='cpu')['state_dict']

In [ ]:
state_dict = {k.replace('model.', ''): v for k, v in pl_state_dict.items()}

In [ ]:
state_dict

In [ ]:
net = SimpleNN(input_size= 28 * 28, hidden_size=128)

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()

In [ ]:
model = InferenceModel(net)

## Visualize weights

In [ ]:
def get_normalized_weights(layer):
    # Get the weights from the first layer
    weights = layer.weight.data.cpu()
    # Reshape weights to (num_neurons, 28, 28)
    weights = weights.view(weights.size(0), 28, 28)
    # Normalize weights to [0, 1]
    min_w = weights.min(dim=1, keepdim=True)[0].min(dim=2, keepdim=True)[0]
    max_w = weights.max(dim=1, keepdim=True)[0].max(dim=2, keepdim=True)[0]
    weights_normalized = (weights - min_w) / (max_w - min_w)
    # Number of neurons in the first layer

    return weights_normalized

In [ ]:
# Visualize the first layer weights after training
def visualize_first_layer_weights(layer):
    import matplotlib.pyplot as plt
    
    weights_normalized = get_normalized_weights(layer)
    # Number of neurons in the first layer
    num_neurons = weights_normalized.size(0)
    # Determine grid size for plotting
    rows = int(num_neurons / 8) + 1
    cols = 8
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
    for i, ax in enumerate(axes.flat):
        if i < num_neurons:
            # Get the normalized weights for the i-th neuron
            w = weights_normalized[i]
            # Display the weights as an image
            ax.imshow(w, cmap='gray')
            ax.set_title(f'Neuron {i}')
            ax.axis('off')
        else:
            ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_first_layer_weights_for_neurons(layer, neurons_list):
    import matplotlib.pyplot as plt

    weights_normalized = get_normalized_weights(layer)
    # Number of neurons to visualize
    num_neurons = len(neurons_list)
    # Determine grid size for plotting
    cols = min(4, num_neurons)
    rows = (num_neurons + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        if i < num_neurons:
            # Get the normalized weights for the i-th neuron
            w = weights_normalized[i]
            # Display the weights as an image
            ax.imshow(w.numpy(), cmap='gray')
            ax.set_title(f'Neuron {neurons_list[i]}')
            ax.axis('off')
        else:
            ax.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_first_layer_weights(net.hidden)

In [ ]:
def visualize_last_layer_weights(layer):
    import matplotlib.pyplot as plt
    import numpy as np

    # Get the weights from the last layer
    last_layer_weights = layer.weight.data.cpu().squeeze()  # Shape: [hidden_size]
    hidden_size = last_layer_weights.size(0)
    # Create an array of neuron indices
    neurons = np.arange(hidden_size)
    # Plot the weights as a bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(neurons, last_layer_weights.numpy())
    plt.xlabel('Hidden Neuron Index')
    plt.ylabel('Weight')
    plt.title('Weights from Hidden Layer to Output Neuron')
    plt.show()
    
    # Optionally, print out the bias
    last_layer_bias = layer.bias.data.cpu().item()
    print(f"Bias of the output neuron: {last_layer_bias:.4f}")

In [ ]:
# Call the visualization function
visualize_last_layer_weights(net.output)

## Load data

In [ ]:
# Updated MNIST data loaders with normalization and validation set
def prepare_data(batch_size=128):
    # Normalize to [0, 1] for MNIST
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.1307,), std=(0.3081,)),  # Mean and std from MNIST
        # transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
    ])

    # Training set
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Validation set
    val_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_dataset, train_loader, val_dataset, val_loader

In [ ]:
train_dataset, train_loader, val_dataset, val_loader = prepare_data()

## Prepare embeddings

In [ ]:
train_y = np.array([y for _, y in train_dataset])
val_y = np.array([y for _, y in val_dataset])

In [ ]:
x_trlabs, z_train = find_zs(train_dataset)
x_vllabs, z_val = find_zs(val_dataset)

In [ ]:
train_z = inference_ds(train_dataset)
val_z = inference_ds(val_dataset)

In [ ]:
train_z

In [ ]:
pos_idx = np.where(net.output.weight.data.cpu().numpy() >= 0)[1]
neg_idx = np.where(net.output.weight.data.cpu().numpy() < 0)[1]
neg_idx, pos_idx

In [ ]:
z = z_train[1][0] 
x = x_trlabs[1][0]

In [ ]:
x = x_trlabs[1][0]
model.inference_np(x.unsqueeze(0))

In [ ]:
z_en = model.run_encode(x)

In [ ]:
z_en.shape[0]

In [ ]:
with torch.inference_mode():
    z_en[0][neg_idx] = 0

In [ ]:
z_en[0][neg_idx]

In [ ]:
y_np = model.regress_np(z_en)
y_np

In [ ]:
# Visualize the first layer weights for the specified neurons
visualize_first_layer_weights_for_neurons(net.hidden, pos_idx)